In [3]:
!pip install gradio_client
from IPython.display import clear_output
clear_output()

In [ ]:
#!unzip -a '/content/drive/MyDrive/Colab Notebooks/张本智和输阿鲁纳：乒乓球没有应该.zip' -d /content/drive/MyDrive/o_videos

In [ ]:
import pickle
api_urls = [
"https://1205e58c08536ffff6.gradio.live",
"https://39c23de3e4db040c6a.gradio.live",
"https://9a92c11bc5825e6959.gradio.live",
"https://d10406deed6ca597c7.gradio.live",
"https://9234f9ec3ac937b598.gradio.live"
]

api_urls = [
"https://3b26065692c5724fb7.gradio.live",
'https://3d451b49b9b090a595.gradio.live',
'https://d81b1eb00e702ec469.gradio.live',
'https://be99908e2d0a70bf64.gradio.live',
'https://80533d58bc8c63b2f2.gradio.live',
]

with open('/content/api_urls.pkl', 'wb') as file:
    pickle.dump(api_urls, file)
!python /content/drive/MyDrive/sub_api.py

['https://3b26065692c5724fb7.gradio.live', 'https://3d451b49b9b090a595.gradio.live', 'https://d81b1eb00e702ec469.gradio.live', 'https://be99908e2d0a70bf64.gradio.live', 'https://80533d58bc8c63b2f2.gradio.live']
[['/content/drive/MyDrive/o_videos/张本智和输阿鲁纳：乒乓球没有应该_076.mp4'], ['/content/drive/MyDrive/o_videos/张本智和输阿鲁纳：乒乓球没有应该_089.mp4']]
Processing 1 files with API URL: https://3b26065692c5724fb7.gradio.live
Loaded as API: https://3b26065692c5724fb7.gradio.live/ ✔
Loaded as API: https://3b26065692c5724fb7.gradio.live/ ✔
Traceback (most recent call last):
  File "/content/drive/MyDrive/process_video_try.py", line 29, in process_video
    result = client.predict(
  File "/usr/local/lib/python3.10/dist-packages/gradio_client/client.py", line 474, in predict
    return self.submit(
  File "/usr/local/lib/python3.10/dist-packages/gradio_client/client.py", line 518, in submit
    args = utils.construct_args(endpoint.parameters_info, args, kwargs)
  File "/usr/local/lib/python3.10/dist-packages/g

In [ ]:
#sub_api.py
import time
import os
import subprocess
from math import ceil
import json
def get_files(directory):
    files = []
    for root, dirs, _files in os.walk(directory):
        for file in _files:
            if file.endswith('.mp4'):
                files.append(os.path.join(root, file))
    return files

def process_videos(directory, api_urls):
    # 获取文件列表
    files = get_files(directory)

    # 平均分配文件
    num_apis = len(api_urls)
    files_per_api = ceil(len(files) / num_apis)
    file_chunks = [files[i:i + files_per_api] for i in range(0, len(files), files_per_api)]

    # 并行处理
    processes = []
    print(file_chunks)
    for i, chunk in enumerate(file_chunks):
        api_url = api_urls[i % num_apis]
        #time.sleep(5)
        print(f"Processing {len(chunk)} files with API URL: {api_url}")
        chunk_str = json.dumps(chunk)
        #print(chunk_str)
        p = subprocess.Popen(['python', '/content/drive/MyDrive/process_video.py', chunk_str, api_url])
        processes.append(p)

    # 等待所有进程完成
    for p in processes:
        p.wait()

# 示例调用
# 定义多个API URL
api_urls = [
"https://c159255c69655a5a35.gradio.live",
"https://dbc4af8ce7406ee36d.gradio.live",
"https://ae206848e75c9c25e8.gradio.live"
]
import pickle
# 从文件中加载变量
with open('/content/api_urls.pkl', 'rb') as file:
    api_urls = pickle.load(file)
print(api_urls)
process_videos('/content/drive/MyDrive/o_videos', api_urls)

In [ ]:
# process_video.py

import os
import requests
import shutil
from gradio_client import Client, handle_file

def download_video(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            shutil.copyfileobj(response.raw, f)
        print(f'Successfully downloaded to {save_path}')
    else:
        print(f'Failed to download from {url}')

def process_video(file_path, api_url):
    client = Client(api_url)
    result = client.predict(
        video_input={"video": handle_file(file_path)},
        api_name="/predict"
    )
    print(result)

    base_url = api_url + '/file=/tmp/gradio/'
    new_result = []
    # 模糊过的视频
    new_result.append(base_url + result[0].split('/')[-1])
    # 遮罩视频
    new_result.append(base_url + result[1].split('/')[-1])
    print(new_result)

    # 创建必要的目录
    '''
    os.makedirs('/content/sam2_output', exist_ok=True)
    os.makedirs('/content/speedup_output', exist_ok=True)
    os.makedirs('/content/mask_output', exist_ok=True)
    '''
    os.makedirs('/content/drive/MyDrive/sam2_output', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/speedup_output', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/mask_output', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/already_videos/', exist_ok=True)
    # 如不是加速，是抠图的
    if "speed_up" not in result[1]:
        print('sam2视频')
        save_path = os.path.join('/content/drive/MyDrive/sam2_output', new_result[0].split('/')[-1])
        download_video(new_result[0], save_path)
        save_path = os.path.join('/content/drive/MyDrive/mask_output', new_result[1].split('/')[-1])
        download_video(new_result[1], save_path)
    else:
        print('加速视频')
        save_path = os.path.join('/content/drive/MyDrive/speedup_output', new_result[0].split('/')[-1])
        download_video(new_result[0], save_path)

    # 移动原始文件到已处理目录
    destination_path = os.path.join('/content/drive/MyDrive/already_videos/', os.path.basename(file_path))
    shutil.move(file_path, destination_path)
    print(f"File moved from '{file_path}' to '{destination_path}'")

if __name__ == "__main__":
    import sys
    import json
# 从命令行参数中获取文件路径列表和API URL
    chunk_str = sys.argv[1]
    api_url = sys.argv[2]

    # 将JSON字符串解析为列表
    chunk = json.loads(chunk_str)
    #print(chunk)
    for file_path in chunk:
      process_video(file_path, api_url)

In [ ]:
# process_video_try.py

import os
import requests
import shutil
from gradio_client import Client, handle_file
import time
import httpx

def download_video(url, save_path):
    try:
        response = requests.get(url, stream=True, timeout=60)  # 增加超时时间
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                shutil.copyfileobj(response.raw, f)
            print(f'Successfully downloaded to {save_path}')
        else:
            print(f'Failed to download from {url}')
    except requests.exceptions.RequestException as e:
        print(f"Download failed: {e}")

def process_video(file_path, api_url, max_retries=3, retry_delay=5):
    client = Client(api_url)
    result = None
    retries = 0

    while retries < max_retries:
        try:
            result = client.predict(
                video_input={"video": handle_file(file_path)},
                api_name="/predict",
                timeout=60  # 增加超时时间
            )
            break
        except (httpx.ReadTimeout, httpx.ConnectTimeout, httpx.ConnectError) as e:
            print(f"Prediction failed: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(retry_delay)
            else:
                print("Max retries reached. Giving up.")
                return
        except Exception as e:
            print(f"Unexpected error: {e}")
            return

    if result is None:
        print("Failed to get prediction result after maximum retries.")
        return

    print(result)

    base_url = api_url + '/file=/tmp/gradio/'
    new_result = []
    # 模糊过的视频
    new_result.append(base_url + result[0].split('/')[-1])
    # 遮罩视频
    new_result.append(base_url + result[1].split('/')[-1])
    print(new_result)

    # 创建必要的目录
    os.makedirs('/content/drive/MyDrive/sam2_output', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/speedup_output', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/mask_output', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/already_videos/', exist_ok=True)

    # 如不是加速，是抠图的
    if "speed_up" not in result[1]:
        print('sam2视频')
        save_path = os.path.join('/content/drive/MyDrive/sam2_output', new_result[0].split('/')[-1])
        download_video(new_result[0], save_path)
        save_path = os.path.join('/content/drive/MyDrive/mask_output', new_result[1].split('/')[-1])
        download_video(new_result[1], save_path)
    else:
        print('加速视频')
        save_path = os.path.join('/content/drive/MyDrive/speedup_output', new_result[0].split('/')[-1])
        download_video(new_result[0], save_path)

    # 移动原始文件到已处理目录
    destination_path = os.path.join('/content/drive/MyDrive/already_videos/', os.path.basename(file_path))
    shutil.move(file_path, destination_path)
    print(f"File moved from '{file_path}' to '{destination_path}'")

if __name__ == "__main__":
    import sys
    import json

    # 从命令行参数中获取文件路径列表和API URL
    chunk_str = sys.argv[1]
    api_url = sys.argv[2]

    # 将JSON字符串解析为列表
    chunk = json.loads(chunk_str)
    for file_path in chunk:
        process_video(file_path, api_url)